<a href="https://colab.research.google.com/github/Nacho2904/pasantiacsc/blob/main/regresion_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Regresión logística

In [11]:
!wget https://raw.githubusercontent.com/CSC-CONICET/CURSO-LEARNING1/main/datasets/parcialitos.txt

--2022-09-06 15:13:20--  https://raw.githubusercontent.com/CSC-CONICET/CURSO-LEARNING1/main/datasets/parcialitos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3775 (3.7K) [text/plain]
Saving to: ‘parcialitos.txt.1’

parcialitos.txt.1   100%[===================>]   3.69K  --.-KB/s    in 0s      

2022-09-06 15:13:20 (62.7 MB/s) - ‘parcialitos.txt.1’ saved [3775/3775]



In [15]:
import pandas as pd
import numpy as np
dataset = pd.read_table("parcialitos.txt.1", sep=',',header=None)
dataset.columns = ["nota_primer_parcialito", "nota_segundo_parcialito", "aprobo"]
dataset

,nota_primer_parcialito,nota_segundo_parcialito,aprobo
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1
...,...,...,...
95,83.489163,48.380286,1
96,42.261701,87.103851,1
97,99.315009,68.775409,1
98,55.340018,64.931938,1
